<a href="https://colab.research.google.com/github/IdjiotSandwiches/knn-fer/blob/create-model/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dagshub
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.2/252.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/8

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import tensorflow as tf
# import mlflow
# import dagshub
import os
import pathlib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/"

Mounted at /content/drive
/content/drive/MyDrive


In [3]:
TRAIN_PATH = pathlib.Path('FER2013')
TEST_PATH = pathlib.Path('facial-emotion-recognition')
LABELS = os.listdir(TRAIN_PATH)

In [4]:
K_SIZE = (31,31)
SIGMA = 4
THETA_RANGE = np.arange(0, np.pi, np.pi/32)
LAMBD = 10.0
GAMMA = 0.5
PSI = 0

gabor_params = {
    'ksize': K_SIZE,
    'sigma': SIGMA,
    'lambd': LAMBD,
    'gamma': GAMMA,
    'psi': PSI
}

In [5]:
def gabor_filter(img):
  features = []
  for theta in THETA_RANGE:
    kernel = cv.getGaborKernel(**gabor_params, theta=theta)
    filter = cv.filter2D(img, cv.CV_64F, kernel)
    features.append(filter)

  return np.array(features)

In [6]:
def open_dataset(dir):
  if(dir == 'FER2013'):
    limit = 1500
  else:
    limit = 300

  images, labels = [], []

  for label in LABELS:
    path = f'{dir}/{label}'
    for j, img_path in enumerate(os.listdir(path)):
      img = cv.imread(f'{path}/{img_path}')
      img = cv.resize(img, (48,48))
      img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
      img = img / 255.0
      img = gabor_filter(img)
      print(f'{label} {j} Done!')
      images.append(img)
      labels.append(label)

      if(j == limit):
        break

  return images, labels

In [7]:
def save_filtered_img(images, labels, path):
  images = np.array(images)
  labels = np.array(labels)

  folder_path = f'/content/drive/MyDrive/gabor-filtered-imgs/{path}'
  os.makedirs(folder_path, exist_ok=True)

  np.save(f'{folder_path}/images.npy', images)
  np.save(f'{folder_path}/labels.npy', labels)

  print('Ok!')

In [8]:
train_images, train_labels = open_dataset(TRAIN_PATH)
test_images, test_labels = open_dataset(TEST_PATH)

happy 0 Done!
happy 1 Done!
happy 2 Done!
happy 3 Done!
happy 4 Done!
happy 5 Done!
happy 6 Done!
happy 7 Done!
happy 8 Done!
happy 9 Done!
happy 10 Done!
happy 11 Done!
happy 12 Done!
happy 13 Done!
happy 14 Done!
happy 15 Done!
happy 16 Done!
happy 17 Done!
happy 18 Done!
happy 19 Done!
happy 20 Done!
happy 21 Done!
happy 22 Done!
happy 23 Done!
happy 24 Done!
happy 25 Done!
happy 26 Done!
happy 27 Done!
happy 28 Done!
happy 29 Done!
happy 30 Done!
happy 31 Done!
happy 32 Done!
happy 33 Done!
happy 34 Done!
happy 35 Done!
happy 36 Done!
happy 37 Done!
happy 38 Done!
happy 39 Done!
happy 40 Done!
happy 41 Done!
happy 42 Done!
happy 43 Done!
happy 44 Done!
happy 45 Done!
happy 46 Done!
happy 47 Done!
happy 48 Done!
happy 49 Done!
happy 50 Done!
happy 51 Done!
happy 52 Done!
happy 53 Done!
happy 54 Done!
happy 55 Done!
happy 56 Done!
happy 57 Done!
happy 58 Done!
happy 59 Done!
happy 60 Done!
happy 61 Done!
happy 62 Done!
happy 63 Done!
happy 64 Done!
happy 65 Done!
happy 66 Done!
happy

In [9]:
save_filtered_img(train_images, train_labels, 'train')
save_filtered_img(test_images, test_labels, 'test')

Ok!
Ok!
